# Week 3 - Data Understanding and Preparation (Part 1)

## Notes

### Importing gapminder

In [2]:
# Clear all the variables in the workspace
rm(list=ls())

# Install the tidyverse package
if (!('tidyverse' %in% installed.packages())) {
    install.packages('tidyverse')
}

# Load the relevant packages into the workspace
suppressWarnings(library('tidyverse')) # For tidyr and readr
suppressWarnings(library('readr')) # To import data (belongs to tidyverse)
suppressWarnings(library('tidyr')) # To tidy data (belongs to tidyverse)
suppressWarnings(library('psych')) # To tidy data (belongs to tidyverse)

gapminder.csv:

In [2]:
gap <- read_csv('data/01/Case Study 1 - Gapminder/gapminder.csv', col_types=cols())
print(dim(gap))
head(gap, 3)

[1] 1704    6


country,year,pop,continent,lifeExp,gdpPercap
Afghanistan,1952,8425333,Asia,28.801,779.4453
Afghanistan,1957,9240934,Asia,30.332,820.8530
Afghanistan,1962,10267083,Asia,31.997,853.1007


gapminder_wide.csv:

In [3]:
gapminder_wide.csv <- read_csv('data/01/Case Study 1 - Gapminder/gapminder_wide.csv', col_types=cols())
print(dim(gapminder_wide.csv))
head(gapminder_wide.csv, 3)

[1] 142  38


continent,country,gdpPercap_1952,gdpPercap_1957,gdpPercap_1962,gdpPercap_1967,gdpPercap_1972,gdpPercap_1977,gdpPercap_1982,gdpPercap_1987,...,pop_1962,pop_1967,pop_1972,pop_1977,pop_1982,pop_1987,pop_1992,pop_1997,pop_2002,pop_2007
Africa,Algeria,2449.008,3013.9760,2550.8169,3246.992,4182.664,4910.417,5745.160,5681.359,...,11000948,12760499,14760787,17152804,20033753,23254956,26298373,29072015,31287142,33333216
Africa,Angola,3520.610,3827.9405,4269.2767,5522.776,5473.288,3008.647,2756.954,2430.208,...,4826015,5247469,5894858,6162675,7016384,7874230,8735988,9875024,10866106,12420476
Africa,Benin,1062.752,959.6011,949.4991,1035.831,1085.797,1029.161,1277.898,1225.856,...,2151895,2427334,2761407,3168267,3641603,4243788,4981671,6066080,7026113,8078314


### Looking more closely at `gapminder_wide.csv`

`gapminder_wide.csv` is not in tidy format which requires that:
1. Each variable forms a columns
1. Each observation forms a row
1. Each type of observational unit forms a table

So, we need to apply some operations to manipulate the data to get it the same as `gapminder.csv` (1706 x 6) which satisfies the tidy format.

First, we need to pivot every column (except continent and country) into a row. We can do this using the `gather()` functions. This gathers the rest of the columns and collapses them into key-value pairs while duplicating all other columns.

In [4]:
gap_long <- gapminder_wide.csv %>%
    gather(key='key', value='value', col=-c(continent, country))
print(dim(gap_long))
head(gap_long, 3)

[1] 5112    4


continent,country,key,value
Africa,Algeria,gdpPercap_1952,2449.008
Africa,Angola,gdpPercap_1952,3520.610
Africa,Benin,gdpPercap_1952,1062.752


Next, we need to separate the `key` into `indicator` and `year`.

In [5]:
gap_long <- gap_long %>%
    separate(col=key, into=c('indicator', 'year'), sep='_')
print(dim(gap_long))
head(gap_long, 3)

[1] 5112    5


continent,country,indicator,year,value
Africa,Algeria,gdpPercap,1952,2449.008
Africa,Angola,gdpPercap,1952,3520.610
Africa,Benin,gdpPercap,1952,1062.752


Finally, we need to pivot the `indicator` column to become columns (i.e. variables). After this, `gap3` has been manipulated to become the same as `gapminder.csv`. We can also confirm they now have the same dimensions.

In [6]:
gap_long <- gap_long %>%
    spread(key='indicator', value='value')
print(dim(gap_long))
head(gap_long, 3)

[1] 1704    6


continent,country,year,gdpPercap,lifeExp,pop
Africa,Algeria,1952,2449.008,43.077,9279525
Africa,Algeria,1957,3013.976,45.685,10270856
Africa,Algeria,1962,2550.817,48.303,11000948


Here's an example of how the `unite()` function works. It's pretty self-explanatory.

In [7]:
head(gap_long, 3) %>%
    unite(col='country_continent', selected_cols=c(country, continent), sep='_')

country_continent,year,gdpPercap,lifeExp,pop
Algeria_Africa,1952,2449.008,43.077,9279525
Algeria_Africa,1957,3013.976,45.685,10270856
Algeria_Africa,1962,2550.817,48.303,11000948


### Assessing the data

In [8]:
summary(gap)

   country               year           pop             continent        
 Length:1704        Min.   :1952   Min.   :6.001e+04   Length:1704       
 Class :character   1st Qu.:1966   1st Qu.:2.794e+06   Class :character  
 Mode  :character   Median :1980   Median :7.024e+06   Mode  :character  
                    Mean   :1980   Mean   :2.960e+07                     
                    3rd Qu.:1993   3rd Qu.:1.959e+07                     
                    Max.   :2007   Max.   :1.319e+09                     
    lifeExp        gdpPercap       
 Min.   :23.60   Min.   :   241.2  
 1st Qu.:48.20   1st Qu.:  1202.1  
 Median :60.71   Median :  3531.8  
 Mean   :59.47   Mean   :  7215.3  
 3rd Qu.:70.85   3rd Qu.:  9325.5  
 Max.   :82.60   Max.   :113523.1  

In [9]:
cols <- c('year', 'pop', 'lifeExp', 'gdpPercap')
describe(gap[,cols])

,vars,n,mean,sd,median,trimmed,mad,min,max,range,skew,kurtosis,se
year,1,1704,1.979500e+03,1.726533e+01,1979.5000,1.979500e+03,2.223900e+01,1952.0000,2.007000e+03,5.500000e+01,0.0000000,-1.218876,4.182540e-01
pop,2,1704,2.960121e+07,1.061579e+08,7023595.5000,1.139946e+07,7.841474e+06,60011.0000,1.318683e+09,1.318623e+09,8.3255400,77.621441,2.571683e+06
lifeExp,3,1704,5.947444e+01,1.291711e+01,60.7125,5.991524e+01,1.610104e+01,23.5990,8.260300e+01,5.900400e+01,-0.2522575,-1.129098,3.129179e-01
gdpPercap,4,1704,7.215327e+03,9.857455e+03,3531.8470,5.221443e+03,4.007608e+03,241.1659,1.135231e+05,1.132820e+05,3.8434960,27.395995,2.387976e+02


In [10]:
str(gap) # Structure of the data

Classes 'spec_tbl_df', 'tbl_df', 'tbl' and 'data.frame':	1704 obs. of  6 variables:
 $ country  : chr  "Afghanistan" "Afghanistan" "Afghanistan" "Afghanistan" ...
 $ year     : num  1952 1957 1962 1967 1972 ...
 $ pop      : num  8425333 9240934 10267083 11537966 13079460 ...
 $ continent: chr  "Asia" "Asia" "Asia" "Asia" ...
 $ lifeExp  : num  28.8 30.3 32 34 36.1 ...
 $ gdpPercap: num  779 821 853 836 740 ...
 - attr(*, "spec")=
  .. cols(
  ..   country = col_character(),
  ..   year = col_double(),
  ..   pop = col_double(),
  ..   continent = col_character(),
  ..   lifeExp = col_double(),
  ..   gdpPercap = col_double()
  .. )


In [11]:
glimpse(gap) # Cleaner version of str

Observations: 1,704
Variables: 6
$ country   <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan...
$ year      <dbl> 1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 199...
$ pop       <dbl> 8425333, 9240934, 10267083, 11537966, 13079460, 14880372,...
$ continent <chr> "Asia", "Asia", "Asia", "Asia", "Asia", "Asia", "Asia", "...
$ lifeExp   <dbl> 28.801, 30.332, 31.997, 34.020, 36.088, 38.438, 39.854, 4...
$ gdpPercap <dbl> 779.4453, 820.8530, 853.1007, 836.1971, 739.9811, 786.113...


In [12]:
class(gap$country) # Class of the vector

[1] "character"

In [13]:
unique(gap$continent)

[1] "Asia"     "Europe"   "Africa"   "Americas" "Oceania"

In [14]:
table(gap$continent) # Think cross-tabs in SPSS


  Africa Americas     Asia   Europe  Oceania 
     624      300      396      360       24 

In [15]:
prop.table(table(gap$continent)) # Proportions


    Africa   Americas       Asia     Europe    Oceania 
0.36619718 0.17605634 0.23239437 0.21126761 0.01408451 

In [16]:
cor(gap[,cols], use='pairwise.complete.obs') # Correlation matrix
# use='pairwise.complete.obs' will only look at complete observations (ignoring NA)

,year,pop,lifeExp,gdpPercap
year,1.00000000,0.08230808,0.43561122,0.22731807
pop,0.08230808,1.00000000,0.06495537,-0.02559958
lifeExp,0.43561122,0.06495537,1.00000000,0.58370622
gdpPercap,0.22731807,-0.02559958,0.58370622,1.00000000


## Practice Questions

### Question 1

Reshape the dataset to tidy form and convert the necessary variables to numeric.

In [17]:
fertility <- read_csv('data/03/fertility_two_countries.csv', col_types=cols())
fertility

country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Germany,2.41,2.44,2.47,2.49,2.49,2.48,2.44,2.37,2.28,...,1.36,1.36,1.37,1.38,1.39,1.40,1.41,1.42,1.43,1.44
South Korea,6.16,5.99,5.79,5.57,5.36,5.16,4.99,4.85,4.73,...,1.20,1.21,1.23,1.25,1.27,1.29,1.30,1.32,1.34,1.36


In [18]:
tidyFertility <- fertility %>%
    gather(key='year', value='value', selected_cols=-country) %>%
    spread(key='country', value='value')
head(tidyFertility, 3)

year,Germany,South Korea
1960,2.41,6.16
1961,2.44,5.99
1962,2.47,5.79


Now, that we've restructured the data, let's take a look at its structure.

In [19]:
glimpse(tidyFertility)

Observations: 56
Variables: 3
$ year          <chr> "1960", "1961", "1962", "1963", "1964", "1965", "1966...
$ Germany       <dbl> 2.41, 2.44, 2.47, 2.49, 2.49, 2.48, 2.44, 2.37, 2.28,...
$ `South Korea` <dbl> 6.16, 5.99, 5.79, 5.57, 5.36, 5.16, 4.99, 4.85, 4.73,...


We notice that `year` is a character vector, so we need to convert that to numeric.

In [20]:
tidyFertility$year = as.numeric(tidyFertility$year)
glimpse(tidyFertility)

Observations: 56
Variables: 3
$ year          <dbl> 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968,...
$ Germany       <dbl> 2.41, 2.44, 2.47, 2.49, 2.49, 2.48, 2.44, 2.37, 2.28,...
$ `South Korea` <dbl> 6.16, 5.99, 5.79, 5.57, 5.36, 5.16, 4.99, 4.85, 4.73,...


### Question 2

In [21]:
lifeExp <- read_csv('data/03/life_expectancy_and_fertility.csv', col_types=cols())
head(lifeExp, 3)

country,1960_fertility,1960_life_expectancy,1961_fertility,1961_life_expectancy,1962_fertility,1962_life_expectancy,1963_fertility,1963_life_expectancy,1964_fertility,...,2011_fertility,2011_life_expectancy,2012_fertility,2012_life_expectancy,2013_fertility,2013_life_expectancy,2014_fertility,2014_life_expectancy,2015_fertility,2015_life_expectancy
Germany,2.41,69.26,2.44,69.85,2.47,70.01,2.49,70.10,2.49,...,1.40,80.5,1.41,80.6,1.42,80.7,1.43,80.7,1.44,80.8
South Korea,6.16,53.02,5.99,53.75,5.79,54.51,5.57,55.27,5.36,...,1.29,80.6,1.30,80.7,1.32,80.9,1.34,80.9,1.36,81.0


Similarly, we reshape the data into tidy form.

In [22]:
tidyLifeExp <- lifeExp %>%
    gather(key='key', value='value', selected_cols=-country) %>%
    separate(col='key', into=c('year', 'indicator'), extra='merge') %>%
    spread(key='indicator', value='value')
head(tidyLifeExp, 3)

country,year,fertility,life_expectancy
Germany,1960,2.41,69.26
Germany,1961,2.44,69.85
Germany,1962,2.47,70.01


### Question 3

Using the `gapminder.csv` and `gapminder_wide.csv`, check that the resulting total sums of columns: `pop`, `lifeExp`, and `gdpPercap` are the same.

In [23]:
paste('Difference between pop:', sum(gap$pop) - sum(gap_long$pop))
paste('Difference between lifeExp:', sum(gap$lifeExp) - sum(gap_long$lifeExp))
paste('Difference between gdpPercap:', sum(gap$gdpPercap) - sum(gap_long$gdpPercap))

[1] "Difference between pop: 0"

[1] "Difference between lifeExp: 0"

[1] "Difference between gdpPercap: 0"

### Question 4

In [24]:
glimpse(gap)

Observations: 1,704
Variables: 6
$ country   <chr> "Afghanistan", "Afghanistan", "Afghanistan", "Afghanistan...
$ year      <dbl> 1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 199...
$ pop       <dbl> 8425333, 9240934, 10267083, 11537966, 13079460, 14880372,...
$ continent <chr> "Asia", "Asia", "Asia", "Asia", "Asia", "Asia", "Asia", "...
$ lifeExp   <dbl> 28.801, 30.332, 31.997, 34.020, 36.088, 38.438, 39.854, 4...
$ gdpPercap <dbl> 779.4453, 820.8530, 853.1007, 836.1971, 739.9811, 786.113...


### Question 5

From the `titanic` package, import the `titanic_train` dataset and state whether the following variables are ordinal categorical, non-ordinal categorical, continuous, or discrete:

| Variable | Type
| :--------| :---
| Survived | Non-ordinal categorical
| Pclass | Ordinal categorical
| Sex | Non-ordingal categorical
| SibSp | Discrete
| Parch | Discrete
| Fare | Continuous

In [25]:
suppressWarnings(library('titanic'))
data(titanic_train)
glimpse(titanic_train)

Observations: 891
Variables: 12
$ PassengerId <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, ...
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0...
$ Pclass      <int> 3, 1, 3, 1, 3, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 3, 2, 3...
$ Name        <chr> "Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley ...
$ Sex         <chr> "male", "female", "female", "female", "male", "male", "...
$ Age         <dbl> 22, 38, 26, 35, 35, NA, 54, 2, 27, 14, 4, 58, 20, 39, 1...
$ SibSp       <int> 1, 1, 0, 1, 0, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 4, 0, 1...
$ Parch       <int> 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 5, 0, 0, 1, 0, 0...
$ Ticket      <chr> "A/5 21171", "PC 17599", "STON/O2. 3101282", "113803", ...
$ Fare        <dbl> 7.2500, 71.2833, 7.9250, 53.1000, 8.0500, 8.4583, 51.86...
$ Cabin       <chr> "", "C85", "", "C123", "", "", "E46", "", "", "", "G6",...
$ Embarked    <chr> "S", "C", "S", "S", "S", "Q", "S", "S", "S", "C", "S", ...


### Question 6

#### Gapminder

Find the number of distinct countries and the last year of data available in the `gapminderall.csv` dataset:

In [26]:
gap_all <- read_csv('data/01/Case Study 1 - Gapminder/gapminderall.csv', col_types=cols())
head(gap_all, 3)

paste('There are', length(unique(gap_all$country)), 'distinct countries in the gapminderall.csv dataset.')
paste('The last year of data available in the gapminderall.csv dataset is', max(gap_all$year))

country,year,infant_mortality,life_expectancy,fertility,population,gdp,continent,region
Albania,1960,115.4,62.87,6.19,1636054,NA,Europe,Southern Europe
Algeria,1960,148.2,47.50,7.65,11124892,13828152297,Africa,Northern Africa
Angola,1960,208.0,35.98,7.32,5270844,NA,Africa,Middle Africa


[1] "There are 185 distinct countries in the gapminderall.csv dataset."

[1] "The last year of data available in the gapminderall.csv dataset is 2016"

#### CAS Motors

Calculate the proportion of  policies for each vehicle brand. Which vehicle brand has the highest proportion and what was it's proportion? And what was the proportion of policies coming from Area F?

In [27]:
casMotor <- read_csv('data/01/Case Study 2 - CAS Motor/freMTPL2freq.csv', col_types=cols())
head(casMotor, 3)

IDpol,ClaimNb,Exposure,Area,VehPower,VehAge,DrivAge,BonusMalus,VehBrand,VehGas,Density,Region
1,1,0.10,D,5,0,55,50,B12,Regular,1217,R82
3,1,0.77,D,5,0,55,50,B12,Regular,1217,R82
5,1,0.75,B,6,2,52,50,B12,Diesel,54,R22


In [28]:
# Proportion of claims by vehicle brand (VehBrand)
options(digits=2)
prop <- c()
for (brand in unique(casMotor$VehBrand)){
    val <- length(which(casMotor$VehBrand==brand)) / length(casMotor$VehBrand)
    prop <- c(prop, val)
}
vehicles <- data.frame(VehBrand=unique(casMotor$VehBrand), prop=prop)
vehicles[order(vehicles$prop, decreasing=TRUE),]

# Proportion of claims from Area F
print(round(length(which(casMotor$Area=='F')) / length(casMotor$Area), 3))

,VehBrand,prop
1,B12,0.245
10,B1,0.240
4,B2,0.236
3,B3,0.079
5,B5,0.051
2,B6,0.042
9,B4,0.037
6,B10,0.026
11,B11,0.020
8,B13,0.018


[1] 0.026


* The vehicle brand with the highest proportion of claims was `B12` which made up 24.5% of all claims.
* The proportion of claims from Area `F` was 2.6%.

#### Big Bank

Recall from the Week 1 Learning Guide that `Class` is the (response) variable where 1 denotes fraud and 0 denotes no fraud.

In [29]:
banks <- read_csv('data/01/Case Study 3 - Big Bank/creditcardcut.csv', col_types=cols())
head(banks, 3)
fraud <- length(which(banks$Class==1))
total <- length(banks$Class)
print(fraud)
print(total)
print(fraud/total)

Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Id
73,1.16,1.25,-1.58,1.48,1.138,-1.020,0.638,-0.14,-0.81,...,-0.23,-0.20,-0.2898,0.78,-0.28,0.0567,0.085,1.0,0,1
128,-0.55,0.91,1.60,1.30,0.588,0.032,0.754,-0.31,-0.27,...,0.42,-0.27,0.1296,-0.23,-0.30,-0.1910,-0.087,10.0,0,2
155,1.17,0.31,0.31,0.52,-0.058,-0.259,-0.044,0.04,-0.34,...,-0.54,0.14,-0.0017,0.14,0.10,-0.0054,0.019,1.3,0,3


[1] 492
[1] 9840
[1] 0.05


Out of the 9,840 transactions, 5% (492) have been fraudulent transactions.

## Resources

* [Suppressing `col_types` messages](https://github.com/tidyverse/readr/pull/1000)